# Make figures for a paper
##### author: Elizabeth A. Barnes, Randal J. Barnes and Mark DeMaria

Will need to run the following:
* _analyze_models.ipynb
* _summarize_metrics.ipynb

In [1]:
import datetime
import os
import pickle
import pprint
import time

import experiment_settings
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import shash
from build_data import build_hurricane_data
import build_model
import model_diagnostics
from silence_tensorflow import silence_tensorflow
import prediction
from sklearn.neighbors import KernelDensity
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

silence_tensorflow()
dpiFig = 400

In [ ]:
__author__ = "Randal J Barnes and Elizabeth A. Barnes"
__version__ = "15 March 2022"

EXP_NAME_LIST = (
                 "intensity40_EPCP72",
                 "intensity41_EPCP72",
                 "intensity42_EPCP72",
                 "intensity43_EPCP72",    
                )
APPEND_NAME = '_testYear_2018_10x10'

# EXP_NAME_LIST = (
#                  "intensity30_EPCP72",
#                  "intensity31_EPCP72",
#                  "intensity32_EPCP72",
#                  "intensity33_EPCP72",    
#                 )
# APPEND_NAME = '_testYear_2018_5x5'

# EXP_NAME_LIST = (
#                  "intensity20_EPCP72",
#                  "intensity21_EPCP72",
#                  "intensity22_EPCP72",
#                  "intensity23_EPCP72",    
#                 )
# APPEND_NAME = '_testYear_2019'

# EXP_NAME_LIST = (
#                  "intensity0_EPCP72",
#                  "intensity1_EPCP72",
#                  "intensity2_EPCP72",
#                  "intensity3_EPCP72",    
#                 )
# APPEND_NAME = '_testYear_2018'

# EXP_NAME_LIST = (
#                  "intensity10_EPCP72",
#                  "intensity11_EPCP72",
#                  "intensity12_EPCP72",
#                  "intensity13_EPCP72",    
#                 )
# APPEND_NAME = '_testYear_2017'


OVERWRITE_METRICS = False
DATA_PATH = "data/"
MODEL_PATH = "saved_models/"
METRIC_PATH = "model_metrics/"
FIGURE_PATH = "figures/summary_plots/"

In [ ]:
mpl.rcParams["figure.facecolor"] = "white"
mpl.rcParams["figure.dpi"] = 150
np.warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

## Define get_metrics()

In [ ]:
def get_metrics():
    tf.random.set_seed(network_seed)
    shash_incs = np.arange(-160,161,1)

    if settings["uncertainty_type"] in ("bnn", "mcdrop", "reg"):       
        # loop through runs for bnn calculation    
        runs = 5_000
        bins_plot = np.linspace(np.min(shash_incs), np.max(shash_incs), 1000)
        bnn_cpd = np.zeros((np.shape(x_test)[0],runs))
        bnn_mode = np.zeros((np.shape(x_test)[0],))

        for i in tqdm(range(0,runs)):
            if settings["uncertainty_type"] == "bnn":
                bnn_cpd[:,i] = np.reshape(model.predict(x_test),np.shape(bnn_cpd)[0])
            elif settings["uncertainty_type"] in ("mcdrop", "reg"):
                bnn_cpd[:,i] = np.reshape(model(x_test,training=True),np.shape(bnn_cpd)[0])                
            else:
                raise NotImplementedError
                
        bnn_mean = np.mean(bnn_cpd,axis=1)
        bnn_median = np.median(bnn_cpd,axis=1)

        for j in tqdm(range(0,np.shape(bnn_mode)[0])):
            kde = KernelDensity(kernel="gaussian", bandwidth=4.).fit(bnn_cpd[j,:].reshape(-1,1))
            log_dens = kde.score_samples(bins_plot.reshape(-1,1))
            i = np.argmax(log_dens)
            bnn_mode[j] = bins_plot[i]

        mean_error, median_error, mode_error = model_diagnostics.compute_errors(onehot_test, bnn_mean, bnn_median, bnn_mode)         
        bins, hist_bnn, pit_D, EDp_bnn = model_diagnostics.compute_pit('bnn',onehot_test, bnn_cpd)
        iqr_capture = model_diagnostics.compute_interquartile_capture('bnn',onehot_test, bnn_cpd)
        iqr_error_spearman, iqr_error_pearson = model_diagnostics.compute_iqr_error_corr('bnn',
                                                                                          onehot_data=onehot_test, 
                                                                                          bnn_cpd=bnn_cpd, 
                                                                                          pred_median=bnn_median,
                                                                                         )
        
    elif settings["uncertainty_type"] in ("shash","shash2", "shash3", "shash4"):         
        shash_cpd = np.zeros((np.shape(x_test)[0],len(shash_incs)))
        shash_mean = np.zeros((np.shape(x_test)[0],))
        shash_med = np.zeros((np.shape(x_test)[0],))
        shash_mode = np.zeros((np.shape(x_test)[0],))

        # loop through samples for shash calculation and get PDF for each sample
        for j in tqdm(range(0,np.shape(shash_cpd)[0])):
            mu_pred, sigma_pred, gamma_pred, tau_pred = prediction.params( x_test[np.newaxis,j], model )
            shash_cpd[j,:] = shash.prob(shash_incs, mu_pred, sigma_pred, gamma_pred, tau_pred)    
            shash_mean[j]  = shash.mean(mu_pred,sigma_pred,gamma_pred,tau_pred)#np.sum(shash_cpd[j,:]*shash_incs)
            shash_med[j]   = shash.median(mu_pred,sigma_pred,gamma_pred,tau_pred)

            i = np.argmax(shash_cpd[j,:])
            shash_mode[j]  = shash_incs[i]

        mean_error, median_error, mode_error = model_diagnostics.compute_errors(onehot_test, shash_mean, shash_med, shash_mode)    
        bins, hist_shash, pit_D, EDp_shash = model_diagnostics.compute_pit('shash',onehot_test, x_data=x_test,model_shash=model)
        iqr_capture = model_diagnostics.compute_interquartile_capture('shash',onehot_test, x_data=x_test,model_shash=model)
        iqr_error_spearman, iqr_error_pearson = model_diagnostics.compute_iqr_error_corr('shash',
                                                                                                onehot_data=onehot_test,
                                                                                                pred_median=shash_med,
                                                                                                x_data=x_test,
                                                                                                model_shash=model,
                                                                                               )
    else:
        raise NotImplementedError
        
    # by definition Consensus is a correction of zero
    cons_error = np.mean(np.abs(0.0 - onehot_test[:,0]))
        
    return mean_error,median_error,mode_error,pit_D,iqr_capture,cons_error,iqr_error_spearman,iqr_error_pearson


## Evaluate the models

In [ ]:
# raise ValueError('do not compute metrics yet')

In [ ]:
import imp
imp.reload(model_diagnostics)

for exp_name in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(exp_name)
    RNG_SEED_LIST = np.copy(settings['rng_seed'])

    for rng_seed in RNG_SEED_LIST:
        settings['rng_seed'] = rng_seed
        NETWORK_SEED_LIST = [settings["rng_seed"]]
        network_seed = NETWORK_SEED_LIST[0]
        tf.random.set_seed(network_seed)  # This sets the global random seed.    

        #----------------------------------------------------------------------------------------------------
        # get the data
        (
            data_summary,
            x_train,
            onehot_train,
            x_val,
            onehot_val,
            x_test,
            onehot_test,    
            df_train,
            df_val,
            df_test,
        ) = build_hurricane_data(DATA_PATH, settings, verbose=0)

        #----------------------------------------------------------------------------------------------------
        # get the model
        # Make, compile, and train the model
        tf.keras.backend.clear_session()            
        model = build_model.make_model(
            settings,
            x_train,
            onehot_train,
            model_compile=False,
        )   
        model_name = (
            exp_name + "_" + 
            settings["uncertainty_type"] + '_' + 
            f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
        )
        
        try:
            model.load_weights(MODEL_PATH + model_name + "_weights.h5")
        except:
            print(model_name + ': model does not exist. skipping...')
            continue

        #----------------------------------------------------------------------------------------------------
        # check if the metric filename exists already
        metric_filename = METRIC_PATH + model_name + '_metrics.pkl'              
        if (os.path.exists(metric_filename) and OVERWRITE_METRICS==False):
            # print(metric_filename + ' exists. Skipping...')
            continue
            
        # get metrics and put into a dictionary
        pprint.pprint(model_name)
                
        # compute the metrics
        mean_error, median_error, mode_error, pit_D, iqr_capture, cons_error, iqr_error_spearman,iqr_error_pearson = get_metrics()        
        d = {'uncertainty_type': settings["uncertainty_type"],
             'network_seed': network_seed,
             'rng_seed': settings['rng_seed'],
             'exp_name': exp_name,
             'mean_error': mean_error,
             'median_error': median_error,
             'mode_error': mode_error,
             'cons_error': cons_error,
             'pit_d': pit_D,
             'iqr_capture': iqr_capture,
             'iqr_error_spearman': iqr_error_spearman[0],
             'iqr_error_pearson': iqr_error_pearson[0],             
             'iqr_error_spearman_p': iqr_error_spearman[1],
             'iqr_error_pearson_p': iqr_error_pearson[1],             
        }
        pprint.pprint(d, width=80)  
        df = pd.DataFrame(data=d, index=[0])
        df.to_pickle(metric_filename)

In [ ]:
# raise ValueError('do not plot yet')

In [ ]:
df_metrics = pd.DataFrame()

for exp_name in EXP_NAME_LIST:
    settings = experiment_settings.get_settings(exp_name)
    RNG_SEED_LIST = np.copy(settings['rng_seed'])
    
    for rng_seed in RNG_SEED_LIST:
        settings['rng_seed'] = rng_seed
        NETWORK_SEED_LIST = [settings["rng_seed"]]
        network_seed = NETWORK_SEED_LIST[0]
    
        model_name = (
                exp_name + "_" + settings["uncertainty_type"] + '_' + f"network_seed_{network_seed}_rng_seed_{settings['rng_seed']}"
        )
        
        # load the metric filename
        metric_filename = METRIC_PATH + model_name + '_metrics.pkl'  
        if os.path.exists(metric_filename)==False:
            print(metric_filename + ' DOES NOT exist. Skipping...')
            continue
    
        # pprint.pprint(model_name)
        df = pd.read_pickle(metric_filename)
        df['basin_lead'] = exp_name[exp_name.rfind('_')+1:]
        df['mean_error_reduction'] = df['cons_error']-df['mean_error']
        df['median_error_reduction'] = df['cons_error']-df['median_error']        
        df['mode_error_reduction'] = df['cons_error']-df['mode_error']        
        df_metrics = pd.concat([df_metrics,df])
df_metrics

In [ ]:
import seaborn as sns
colors = ('#284E60','#D95980','#E1A730','#C3B1E1')

x_axis_list = ("basin_lead",)#("basin_lead", "exp_name")

for x_axis in x_axis_list:
    f, axs = plt.subplots(4, 2, figsize=(15,20))
    axs = axs.flatten()
    
    for imetric, metric in enumerate(('mean_error',
                                      'mean_error_reduction',
                                      'median_error',
                                      'median_error_reduction',                                                                        
                                      'iqr_error_spearman',
                                      'mode_error_reduction',                                                                        
                                      # 'iqr_error_pearson',                                  
                                      # 'mode_error',
                                      'pit_d',
                                      'iqr_capture')):
        ax = axs[imetric]
        g1 = sns.boxplot(x=x_axis, 
                         y=metric, 
                         hue="uncertainty_type",
                         data=df_metrics,
                         palette=colors,
                         boxprops={'alpha':.2,
                                   'edgecolor': 'white',
                                  },
                         fliersize=0,
                         ax=ax)
        g2 = sns.swarmplot(x=x_axis, 
                           y=metric, 
                           hue="uncertainty_type",
                           palette=colors,
                           data=df_metrics, 
                           dodge=True,
                           ax=ax)

        if(metric=='iqr_capture'):
            ax.axhline(y=0.5,linewidth=3,linestyle='--',color='gray')
            ax.set_ylim(0,1.0)
        if(metric=='pit_d'):
            ax.set_ylim(0,None)
        if(metric.find('reduction') > -1):
            ax.axhline(y=0.0,linewidth=3,linestyle='--',color='gray')
            ax.set_ylim(-4.,4.)
        if(metric.find('reduction') == -1 and metric.find('n_error') > -1):
            ax.set_ylim(0.,22.)
        if(metric.find('iqr_error')> -1):
            ax.set_ylim(-.3,1.)
            ax.axhline(y=0.0,linewidth=3,linestyle='--',color='gray')        


        ax.set_title(metric + APPEND_NAME)
        ax.legend(fontsize=10,frameon=True)
        ax.set_xticklabels(ax.get_xticklabels(),rotation = 30)

    plt.tight_layout()    
    plt.savefig(FIGURE_PATH + 'comparisonsMetrics' + APPEND_NAME + '_' + x_axis + '.png', dpi=dpiFig)    
    # plt.close()
    plt.show()